# AEROSP 536 Electric Propulsion: Homework 1
Jason Chen

## Problem 3
### Part (a)
We need to check for inconsistencies in the data provided. First, we can check if the efficiency 


In [ ]:
import numpy as np

data = {
    'P_in'     : np.array([4.46, 1.26, 1.25, 3.26, 3.24]) * 1e3,
    'F_t'      : np.array([357.8, 248.5, 264.0, 92.7, 95.0]) * 1e-3,
    'F_t_range': np.array([  4.6,   3.3,   2.7,  2.7,  3.0]) * 1e-3,
    'eta'      : np.array([0.47, 0.39, 0.42, 0.44, 0.49]),
    'eta_range': np.array([0.01, 0.02, 0.03, 0.01, 0.02]),
    'isp'      : np.array([1196, 1086, 1132, 1163, 1235]),
    'isp_range': np.array([  15,   32,   33,   16,   16])
}
num_entries = 5
for i in range(num_entries):
    mdot_1 = data['F_t'][i] / (data['isp'][i] * 9.81)
    mdot_2 = (1/2) * (data['F_t'][i]**2) / (data['eta'][i] * data['P_in'][i])
    print(f"Residual mass flow rate for {i}: {mdot_1 - mdot_2:.3e} kg/s")



Residual 0: -4.060042e-08 kg/s
Specific energy: 146.2 MJ/kg
Residual 1: -3.950766e-05 kg/s
Specific energy: 54.02 MJ/kg
Residual 2: -4.260390e-05 kg/s
Specific energy: 52.58 MJ/kg
Residual 3: 5.129713e-06 kg/s
Specific energy: 401.2 MJ/kg
Residual 4: 4.998952e-06 kg/s
Specific energy: 413.2 MJ/kg
